In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 800)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 600)
#cap.set(cv2.CAP_PROP_FRAME_HEITGHT, 1080)

fox=cv2.imread("madfox.jpg")
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(fox,None)

def detect_features(img,kp1,des1,frame,frame_g):
    MIN_MATCH_COUNT = 10
    
    kp2, des2 = sift.detectAndCompute(frame_g,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # Configura o algoritmo de casamento de features
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    circles = None
    circles=cv2.HoughCircles(frame_g,cv2.HOUGH_GRADIENT,2,60,param1=200,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            print(i)
            cv2.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(frame,(i[0],i[1]),2,(0,0,255),3)

    if len(good)>MIN_MATCH_COUNT and circles is not None:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w = img.shape[0],img.shape[1]
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        frame = cv2.polylines(frame,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        
    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None

    draw_params = dict(matchColor = (0,0,255), # draw matches in green color
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = 2)
    return frame

while(True):
    ret, frame = cap.read()
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    frame_gray=cv2.medianBlur(frame_gray,5)
    #http://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html
    frame=detect_features(fox,kp1, des1,frame,frame_gray)
    # Display the resulting frame
    cv2.imshow('original',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[279 451  45]
[279 453  51]
[487 411  57]
[0 0 0]
[0 0 0]
[295 411  45]
[485 387  54]
[123 321  59]
[463 155  32]
[297 405  42]
[481 385  54]
[131 327  57]
[297 401  43]
[487 381  54]
[297 397  41]
[483 383  54]
[299 399  43]
[487 383  57]
[297 395  42]
[295 397  42]
[483 383  54]
[463 157  32]
[295 401  43]
[485 385  54]
[295 401  41]
[481 381  48]
[293 403  41]
[483 387  57]
[293 403  45]
[485 387  57]
[293 405  45]
[295 401  42]
[481 383  52]
[293 399  42]
[487 383  54]
[299 389  43]
[485 375  54]
[303 375  41]
[487 365  54]
[303 365  40]
[485 351  46]
[305 355  45]
[489 345  52]
[497 333  59]
[313 333  43]
[315 315  39]
[319 297  38]
[495 301  54]
[325 287  36]
[329 283  37]
[333 283  34]
[335 285  36]
[337 289  35]
[333 297  36]
[333 307  34]
[329 311  33]
[329 307  35]
[327 307  36]
[327 309  36]
[325 307  32]
[323 309  32]
[321 309  34]
[321 307  31]
[319 307  33]
[319 309  35]
[317 309  35]
[317 309  34]
[317 311  31]
[325 309  32]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0]
[0 0 0

error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\matmul.cpp:2268: error: (-215) scn + 1 == m.cols in function cv::perspectiveTransform
